In [ ]:
# Parameter inputs
aragorn_submit_url = "https://aragorn-u24.apps.renci.org/robokop/query"
answer_coalescer_submit_url = "https://answercoalesce.renci.org/1.4/coalesce/all"
input_search_string = 'ppara'
output_search_string = 'liver fibrosis'

# Initializing directory to write
from datetime import datetime
from pathlib import Path

now = datetime.now()
dt_string = now.strftime("%Y-%m-%d_%H%M%S")
# write_dir = Path("output/TRAPI_enrichment",str(dt_string))
# write_dir.mkdir(parents=True, exist_ok=True)

In [ ]:
import requests
import json
import pprint
pp = pprint.PrettyPrinter(indent=5)

In [ ]:
results = requests.post(f'https://name-resolution-sri.renci.org/lookup?string={input_search_string}&offset=0&limit=100')
results_json = results.json()
#print(json.dumps(results_json,indent=4))
input_node_id_list = list(results_json.keys())
print(input_node_id_list)


In [ ]:
results = requests.post(f'https://name-resolution-sri.renci.org/lookup?string={output_search_string}&offset=0&limit=100')
results_json = results.json()
#print(json.dumps(results_json,indent=4))
output_node_id_list = list(results_json.keys())
print(output_node_id_list)

In [ ]:
query={
    "message": {
      "query_graph": {
        "edges": {
          "e00": {
            "subject": "n00",
              "object": "n01",
          "predicates":["biolink:related_to"]
          },
          "e01": {
            "subject": "n01",
              "object": "n02",
          "predicates":["biolink:related_to"]
          }
        },
        "nodes": {
          "n00": {
            "ids": ['NCBIGene:5465'], # input_node_id_list
            "categories": ["biolink:GeneOrGeneProduct"]
          },
          "n01": {
              "categories": ["biolink:BiologicalEntity"]
          },
          "n02": {
            "ids": ["HP:0001395"], # output_node_id_list
            "categories": ["biolink:DiseaseOrPhenotypicFeature"]
          }
        }
      }
    }
  }


In [ ]:
response_ara = requests.post(aragorn_submit_url,json=query)
print(response_ara.status_code)
number_pathway_results = len(response_ara.json()['message']['results'])
print(len(response_ara.json()['message']['results']))

In [ ]:
json_to_write = response_ara.json()
json_object = json.dumps(json_to_write, indent=4)
 
# Writing TRAPI response_ara to JSON file
with open("trapi_query_response_ara.json", "w") as outfile:
    outfile.write(json_object)

To run the enrichment script `enrich_view.py`, from the Command Line, run `panel serve enrich_view.py --show --autoreload`

In [ ]:
response_coalescer = requests.post(answer_coalescer_submit_url,json=response_ara.json())
print(response_coalescer.status_code)
print(response_coalescer.json())

In [ ]:
json_to_write = response_coalescer.json()
json_object = json.dumps(json_to_write, indent=4)
 
# Writing TRAPI response_ara to JSON file
with open("trapi_query_answer_coalescer.json", "w") as outfile:
    outfile.write(json_object)